In [1]:
from sage.rings.polynomial.pbori.pbori import BooleSet
import json
import itertools
import pandas as pd
import numpy as np
import functools

Here we create a boolean ring in 12 variables named $B$ in the variables $x_i, y_i$ for $i=1,\dots,6$. We use 6 binary digits to specify an element of $BO$, so we are able to specify two elements simultaenously in this ring. This is required for solving the multiplication table.

In [2]:
B.<x1,x2,x3,x4,x5,x6,y1,y2,y3,y4,y5,y6> = BooleanPolynomialRing(12)

We create a ring named $Q_{\text{params}}$ in 6 variables, over a symbolic ring. These will surve as the parameters to specify a quanternion representation of $BO$.
Using the parameter ring, we create $Q$, a quaternion algebra. The technical details are unimportant; essentially we obtain quaternion functions in $z_i$.

In [3]:
Q_params.<z1,z2,z3,z4,z5,z6> = PolynomialRing(SR, 6)
Q.<i,j,k> = QuaternionAlgebra(Q_params, -1, -1)

u = -1/2 * (1 + i + j + k)
t = (1 + i) / sqrt(2)

The basic idea of our trick is that, we create the polynomials implementing our gate via a table join. Given a sequence of generators, we produce a table pairing each generated element with the corresponding sequence of powers of generators that are needed to produce that element. This gives us a "name" for each element in the group. Then we can produce each element of the group by brute force, and match the input sequence of 12 powers, to our "name" sequence of 6. 

So, first we produce the tables we need to match over. This is arbitrary, and we need only be consistent. We create a variable to hold a dataframe. The rows of the dataframe consist of the element, written in quaternion form, corresponding to the product $$(-1)^{p_1} j^{p_2} k^{p_3} u^{2p_4 + p_5} t^{p_6}$$ followed by the parameters $p_i$ which were used.


In [4]:
canonical_forms = pd.DataFrame(data = [{ "element" : str((-1)**p1 * j**p2 * k**p3 * u**(2*p4 + p5) * t**p6),
                                         "z1" : p1,
                                         "z2" : p2,
                                         "z3" : p3,
                                         "z4" : p4,
                                         "z5" : p5,
                                         "z6" : p6 }
                                       for p1, p2, p3, p4, p5, p6
                                       in itertools.product(range(2),range(2),range(2),range(2),range(2),range(2))])

canonical_forms = canonical_forms[(canonical_forms.z4 + canonical_forms.z5) != 2]



!!Slow Cell!!

The multiplication table is constructed similarly. The only change is that we have a product of two elements, followed by the parameters that produced it.

In [5]:
mult_table = pd.DataFrame([{ "element" : str( ((-1)**p1 * j**p2 * k**p3 * u**(2*p4 + p5) * t**p6) * ((-1)**q1 * j**q2 * k**q3 * u**(2*q4 + q5) * t**q6) ),
                             "x1" : p1,
                             "x2" : p2,
                             "x3" : p3,
                             "x4" : p4,
                             "x5" : p5,
                             "x6" : p6,
                             "y1" : q1,
                             "y2" : q2,
                             "y3" : q3,
                             "y4" : q4,
                             "y5" : q5,
                             "y6" : q6 }
                           for p1, p2, p3, p4, p5, p6, q1, q2, q3, q4, q5, q6
                           in itertools.product(range(2),range(2),range(2),range(2),range(2),range(2),range(2),range(2),range(2),range(2),range(2),range(2))])

mult_table = mult_table[(mult_table.x4 + mult_table.x5) != 2]
mult_table = mult_table[(mult_table.y4 + mult_table.y5) != 2]

!!Slow Cell!!

This is the key step the tables are used in. We simply join them (as in SQL) over the common element; resulting in a relation between the standard representation of an element, and the representation given by the product table. 

In [6]:
mult_table_idens = mult_table.join(canonical_forms.set_index("element"), on = "element")

Our goal is now then to acquire an arithmetic expresion for the relation obtained by the join. Used the PoliBoRi package in Sagemath, we can specify the ones and zeros of the relation, and it can efficiently compute a set of polynomials satisfying it. For instance if we know $(1,1,1,1,1,1,1,1,1,1,1,1)$ and $(1, 0, 0, 1, 0, 0)$ both produce $u^2t^1$, then we need 6 polyomials. The first must give 1 on an input of 12 1s. The second must give 0 on the third input. And so on. Note that this set of polynomials must hold for each and every element of the relation.

Here we prepare the data in the format PoliBoRi can process it. From our joined tables we extract all those elements which are either 0 or 1 for a given position in our name sequence.

In [7]:
z1_zeros = mult_table_idens[mult_table_idens.z1 == 0].loc[:,["x1","x2","x3","x4","x5","x6","y1","y2","y3","y4","y5","y6"]].values
z1_zeros = set(list(map(tuple, z1_zeros)))
z1_ones = mult_table_idens[mult_table_idens.z1 == 1].loc[:,["x1","x2","x3","x4","x5","x6","y1","y2","y3","y4","y5","y6"]].values
z1_ones = set(list(map(tuple, z1_ones)))

z2_zeros = mult_table_idens[mult_table_idens.z2 == 0].loc[:,["x1","x2","x3","x4","x5","x6","y1","y2","y3","y4","y5","y6"]].values
z2_zeros = set(list(map(tuple, z2_zeros)))
z2_ones = mult_table_idens[mult_table_idens.z2 == 1].loc[:,["x1","x2","x3","x4","x5","x6","y1","y2","y3","y4","y5","y6"]].values
z2_ones = set(list(map(tuple, z2_ones)))

z3_zeros = mult_table_idens[mult_table_idens.z3 == 0].loc[:,["x1","x2","x3","x4","x5","x6","y1","y2","y3","y4","y5","y6"]].values
z3_zeros = set(list(map(tuple, z3_zeros)))
z3_ones = mult_table_idens[mult_table_idens.z3 == 1].loc[:,["x1","x2","x3","x4","x5","x6","y1","y2","y3","y4","y5","y6"]].values
z3_ones = set(list(map(tuple, z3_ones)))

z4_zeros = mult_table_idens[mult_table_idens.z4 == 0].loc[:,["x1","x2","x3","x4","x5","x6","y1","y2","y3","y4","y5","y6"]].values
z4_zeros = set(list(map(tuple, z4_zeros)))
z4_ones = mult_table_idens[mult_table_idens.z4 == 1].loc[:,["x1","x2","x3","x4","x5","x6","y1","y2","y3","y4","y5","y6"]].values
z4_ones = set(list(map(tuple, z4_ones)))

z5_zeros = mult_table_idens[mult_table_idens.z5 == 0].loc[:,["x1","x2","x3","x4","x5","x6","y1","y2","y3","y4","y5","y6"]].values
z5_zeros = set(list(map(tuple, z5_zeros)))
z5_ones = mult_table_idens[mult_table_idens.z5 == 1].loc[:,["x1","x2","x3","x4","x5","x6","y1","y2","y3","y4","y5","y6"]].values
z5_ones = set(list(map(tuple, z5_ones)))

z6_zeros = mult_table_idens[mult_table_idens.z6 == 0].loc[:,["x1","x2","x3","x4","x5","x6","y1","y2","y3","y4","y5","y6"]].values
z6_zeros = set(list(map(tuple, z6_zeros)))
z6_ones = mult_table_idens[mult_table_idens.z6 == 1].loc[:,["x1","x2","x3","x4","x5","x6","y1","y2","y3","y4","y5","y6"]].values
z6_ones = set(list(map(tuple, z6_ones)))

PolyBoRi does some magic, and voila: we have the 6 polynomials required.

In [8]:
z1_poly = B.interpolation_polynomial(z1_zeros,z1_ones)
z2_poly = B.interpolation_polynomial(z2_zeros,z2_ones)
z3_poly = B.interpolation_polynomial(z3_zeros,z3_ones)
z4_poly = B.interpolation_polynomial(z4_zeros,z4_ones)
z5_poly = B.interpolation_polynomial(z5_zeros,z5_ones)
z6_poly = B.interpolation_polynomial(z6_zeros,z6_ones)

!!Slow Cell!!


Not so fast! Lets make sure theyre actually right.

First we do some uninportant messageing to that Sagemath likes the types, and define a couple of helper functions. We need one just to map out bit-tuple represerntion to the quaternion represention, and another which just multiplies elements.
Next we iterate through all elements (skipping those cases which are extraneous) and compute the correct result by literally multiplying the elements, and also by looking up the anwers with out polynomials. We collect the results of this comparison into a list and then test everything. If all is well, then you should see "True" print.

In [9]:
BZ =  GF(2)["x1","x2","x3","x4","x5","x6","y1","y2","y3","y4","y5","y6"]
z1_poly = BZ(B.interpolation_polynomial(z1_zeros,z1_ones))
z2_poly = BZ(B.interpolation_polynomial(z2_zeros,z2_ones))
z3_poly = BZ(B.interpolation_polynomial(z3_zeros,z3_ones))
z4_poly = BZ(B.interpolation_polynomial(z4_zeros,z4_ones))
z5_poly = BZ(B.interpolation_polynomial(z5_zeros,z5_ones))
z6_poly = BZ(B.interpolation_polynomial(z6_zeros,z6_ones))

def powers_to_el(p1,p2,p3,p4,p5,p6):
    return ((-1)**p1 * j**p2 * k**p3 * u**(2*p4 + p5) * t**p6)

def multiply_els(p1,p2,p3,p4,p5,p6,q1,q2,q3,q4,q5,q6):
    left_operand = (-1)**p1 * j**p2 * k**p3 * u**(2*p4 + p5) * t**p6
    right_operand = (-1)**q1 * j**q2 * k**q3 * u**(2*q4 + q5) * t**q6
    return left_operand * right_operand


results = []
for p1, p2, p3, p4, p5, p6, q1, q2, q3, q4, q5, q6 in list(itertools.product(range(2), \
                                                                             range(2), \
                                                                             range(2), \
                                                                             range(2), \
                                                                             range(2), \
                                                                             range(2), \
                                                                             range(2), \
                                                                             range(2), \
                                                                             range(2), \
                                                                             range(2), \
                                                                             range(2), \
                                                                             range(2))):



    #Ignore impossible case (where our polys fail!)
    if (p4 and p5) or (q4 and q5):
        continue

    z1_pow = int(z1_poly(p1,p2,p3,p4,p5,p6,q1,q2,q3,q4,q5,q6))
    z2_pow = int(z2_poly(p1,p2,p3,p4,p5,p6,q1,q2,q3,q4,q5,q6))
    z3_pow = int(z3_poly(p1,p2,p3,p4,p5,p6,q1,q2,q3,q4,q5,q6))
    z4_pow = int(z4_poly(p1,p2,p3,p4,p5,p6,q1,q2,q3,q4,q5,q6))
    z5_pow = int(z5_poly(p1,p2,p3,p4,p5,p6,q1,q2,q3,q4,q5,q6))
    z6_pow = int(z6_poly(p1,p2,p3,p4,p5,p6,q1,q2,q3,q4,q5,q6))

    product = multiply_els(p1,p2,p3,p4,p5,p6,q1,q2,q3,q4,q5,q6)
    potential_result = powers_to_el(z1_pow,z2_pow,z3_pow,z4_pow,z5_pow,z6_pow)

    results.append(product == potential_result)

print(all(results))

True


Next, we extract the action of each individual generator on a general element. I.e. For a given element specified by $(y_1,\dots,y_6)$ we wish to produce some new 6-sequence given by polynomials in $y_i$.



In [10]:
# I dont remember what these left and right actions were for
# So I guess I better leave them for now

def left_action(x1,x2,x3,x4,x5,x6):
    return(z1_poly(x1=x1,x2=x2,x3=x3,x4=x4,x5=x5,x6=x6),
           z2_poly(x1=x1,x2=x2,x3=x3,x4=x4,x5=x5,x6=x6),
           z3_poly(x1=x1,x2=x2,x3=x3,x4=x4,x5=x5,x6=x6),
           z4_poly(x1=x1,x2=x2,x3=x3,x4=x4,x5=x5,x6=x6),
           z5_poly(x1=x1,x2=x2,x3=x3,x4=x4,x5=x5,x6=x6),
           z6_poly(x1=x1,x2=x2,x3=x3,x4=x4,x5=x5,x6=x6))


def right_action(y1,y2,y3,y4,y5,y6):
    return(z1_poly(y1=y1,y2=y2,y3=y3,y4=y4,y5=y5,y6=y6),
           z2_poly(y1=y1,y2=y2,y3=y3,y4=y4,y5=y5,y6=y6),
           z3_poly(y1=y1,y2=y2,y3=y3,y4=y4,y5=y5,y6=y6),
           z4_poly(y1=y1,y2=y2,y3=y3,y4=y4,y5=y5,y6=y6),
           z5_poly(y1=y1,y2=y2,y3=y3,y4=y4,y5=y5,y6=y6),
           z6_poly(y1=y1,y2=y2,y3=y3,y4=y4,y5=y5,y6=y6))


def t_action(y1,y2,y3,y4,y5,y6):
    return (y2*y4*y6 + y3*y4*y6 + y2*y5*y6 + y2*y3 + y2*y4 + y3*y5 + y3*y6 + y4*y6 + y1 + y3 + y5,
            y4*y6 + y3 + y5 + y6,
            y5*y6 + y2 + y4 + y5 + y6,
            y5,
            y4,
            y6 + 1)

def u1_action(y1,y2,y3,y4,y5,y6):
    return (y1, y2 + y3, y2, y5, y4 + y5 + 1, y6)

def u2_action(y1,y2,y3,y4,y5,y6):
    return (y1, y3, y2 + y3, y4 + y5 + 1, y4, y6)

def k_action(y1,y2,y3,y4,y5,y6):
    return (y1 + y2 + y3, y2, y3 + 1, y4, y5, y6)

def j_action(y1,y2,y3,y4,y5,y6):
    return (y1 + y2, y2 + 1, y3, y4, y5, y6)

def neg_action(y1,y2,y3,y4,y5,y6):
    return (y1 + 1,y2,y3,y4,y5,y6)

def compose(f, g):
    return lambda *a: f(*g(*a))

Now we can find each of the actions we need to implement each individual gate. These are what we need to actually implement the circuits.

In [11]:
t_action(y1,y2,y3,y4,y5,y6)

(y1 + y2*y3 + y2*y4*y6 + y2*y4 + y2*y5*y6 + y3*y4*y6 + y3*y5 + y3*y6 + y3 + y4*y6 + y5,
 y3 + y4*y6 + y5 + y6,
 y2 + y4 + y5*y6 + y5 + y6,
 y5,
 y4,
 y6 + 1)

In [12]:
u1_action(y1,y2,y3,y4,y5,y6)

(y1, y2 + y3, y2, y5, y4 + y5 + 1, y6)

In [13]:
u2_action(y1,y2,y3,y4,y5,y6)

(y1, y3, y2 + y3, y4 + y5 + 1, y4, y6)

In [14]:
k_action(y1,y2,y3,y4,y5,y6)

(y1 + y2 + y3, y2, y3 + 1, y4, y5, y6)

In [15]:
j_action(y1,y2,y3,y4,y5,y6)

(y1 + y2, y2 + 1, y3, y4, y5, y6)

In [16]:
neg_action(y1,y2,y3,y4,y5,y6)

(y1 + 1, y2, y3, y4, y5, y6)

Dont rememebr what this bit was so Ill leave it for now.

In [17]:
res = []
for (x1,x2,x3,x4,x5,x6) in itertools.product(range(2),range(2),range(2),range(2),range(2),range(2)):
    stack = []
    
    if x4 == 1 and x5 == 1:
        continue
    elif x1 == 0 and x2 == 0 and x3 == 0 and x4 == 0 and x5 == 0 and x6 == 0:
        continue
        
    if x6 == 1:
        stack.insert(0,t_action)
        
    if x5 == 1:
        stack.insert(0,u1_action)
        
    if x4 == 1:
        stack.insert(0,u2_action)
        
    if x3 == 1:
        stack.insert(0,k_action)
        
    if x2 == 1:
        stack.insert(0,j_action)
        
    if x1 == 1:
        stack.insert(0,neg_action) 
        
    action = functools.reduce(compose,stack)
    
    res.append(action(y1,y2,y3,y4,y5,y6) == left_action(x1,x2,x3,x4,x5,x6))
    
all(res)    

False

In [18]:
stack.append(1)

In [19]:
neg_action(y1,y2,y3,y4,y5,y6)

(y1 + 1, y2, y3, y4, y5, y6)